In [1]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Dense, SimpleRNN, RepeatVector
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback

from termcolor import colored

In [2]:
allChars = "90348756921+"

In [3]:
len(allChars)
char_to_index = dict((c, i) for i, c in enumerate(allChars))
index_to_char = dict((i, c) for i, c in enumerate(allChars))

In [4]:
char_to_index

{'+': 11,
 '0': 1,
 '1': 10,
 '2': 9,
 '3': 2,
 '4': 3,
 '5': 6,
 '6': 7,
 '7': 5,
 '8': 4,
 '9': 8}

In [5]:
def numberGenerate():
    firstNumber = np.random.randint(3, 10)
    SecNumber = np.random.randint(3, 10)
    newValue = str(firstNumber) + "+"+str(SecNumber)
    finalValue = str(firstNumber+SecNumber)
    return newValue, finalValue

In [6]:
numberGenerate()

('4+5', '9')

In [7]:
model = Sequential([
    
    SimpleRNN(128, input_shape = (None, len(allChars))),
    RepeatVector(3),
    
    
    SimpleRNN(128, return_sequences = True),
    TimeDistributed(Dense(len(allChars), activation = "softmax"))])

In [8]:
model.compile(loss = "categorical_crossentropy",
             optimizer = "adam", 
             metrics = ["accuracy"])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 128)               18048     
                                                                 
 repeat_vector (RepeatVector  (None, 3, 128)           0         
 )                                                               
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 3, 128)            32896     
                                                                 
 time_distributed (TimeDistr  (None, 3, 12)            1548      
 ibuted)                                                         
                                                                 
Total params: 52,492
Trainable params: 52,492
Non-trainable params: 0
_________________________________________________________________


In [11]:
def vector(newvalue, finalvalue):
    
    x = np.zeros((3, len(allChars)))
    y = np.zeros((3, len(allChars)))
    
    difference_x = 3 - len(newvalue)
    difference_y = 3 - len(finalvalue)
    
    
    for i, c in enumerate(newvalue):
        x[difference_x + i, char_to_index[c]] = 1
        
    for i in range(difference_x):
        x[i, difference_x["0"]] = 1
    
    
    for i, c in enumerate(finalvalue):
        y[difference_y + i, char_to_index[c]] = 1
        
    for i in range(difference_y):
        y[i, char_to_index["0"]] = 1  
    
    return x, y

In [12]:
newValue, finalValue = numberGenerate()

In [13]:
print("New Value: ", newValue)
print("Final Value: ", finalValue)

New Value:  3+5
Final Value:  8


In [14]:
x, y = vector(newValue, finalValue)

In [15]:
print("X Value: ", x)
print("\n")
print("Y Value: ", y)

X Value:  [[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


Y Value:  [[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]


In [16]:
def ReturnVector(value):
    data = [index_to_char[np.argmax(vector)] for i, vector in enumerate(value)]
    return "".join(data)

In [17]:
ReturnVector(x)

'3+5'

In [18]:
ReturnVector(y)

'008'

In [19]:
def trainData_and_TestData(num_samples = 100000):
    
    x_trainData = np.zeros((num_samples, 3, len(allChars)))
    y_trainData = np.zeros((num_samples, 3, len(allChars)))
    
    
    for i in range(num_samples):
        newValue, finalValue = numberGenerate()
        x, y = vector(newValue, finalValue)
        
        x_trainData[i] = x
        y_trainData[i] = y
        
    return x_trainData, y_trainData

In [20]:
x_trainData, y_trainData = trainData_and_TestData()

In [21]:
x_trainData.shape

(100000, 3, 12)

In [22]:
ReturnVector(y_trainData[0])

'011'

In [23]:
Call_backs =  LambdaCallback(
    on_epoch_end = lambda newValue, finalValue: print("{:.2f}".format(finalValue["accuracy"]), end = "|"))

earlyStop = EarlyStopping(monitor = "val_loss", patience=10)

In [24]:
model.fit(x_trainData, 
         y_trainData,
         epochs=10,
         validation_split=0.2, 
         verbose = False, 
         callbacks = [Call_backs, earlyStop])

0.99|1.00|1.00|1.00|1.00|1.00|1.00|1.00|1.00|1.00|

In [25]:
x_test, y_test = trainData_and_TestData(num_samples= 3)
predictedvalue = model.predict(x_test)


sequenceAccuracy = 0

for i, predictedData in enumerate(predictedvalue):
    predData = ReturnVector(predictedData)
    
    y_testData = ReturnVector(y_test[i])
    x_testData = ReturnVector(x_test[i])
    
    #sucess = "green" if predData == y_testData else "red"
    initialP = int(predData == y_testData)
    sequenceAccuracy += 1/len(predData)*initialP
    #print("Pred", predData)
    #print("Initial", initialP)
    
    
    result = "New Value: {}, Final value: {}, PredictedData: {}".format(x_testData, y_testData, predData)
    #print(colored(sucess, result))
    
print("Final Accuracy is : {:.3f}%".format(100*sequenceAccuracy))

Final Accuracy is : 100.000%
